In [1]:
import os
import cv2
import numpy as np

In [2]:
# Define paths
base_dir = './'
prototxt_path = os.path.join(base_dir + 'model_data/deploy.prototxt')
caffemodel_path = os.path.join(base_dir + 'model_data/weights.caffemodel')

#working directory
dir_name = 'pruebas'

raw_images_dir = os.path.join(base_dir + 'dataset/' + dir_name)
extracted_faces_dir = os.path.join(base_dir + 'dataset/new_' + dir_name)

# Read the model
model = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Create directory 'faces' if it does not exist
if not os.path.exists(extracted_faces_dir):
    print("New directory created")
    os.makedirs(extracted_faces_dir)

In [3]:
#Loop through all images and strip out faces
count = 0
iter=0
for file in os.listdir(raw_images_dir):
    file_name, file_extension = os.path.splitext(file)
    if (file_extension in ['.png','.jpg', '.JPG', '.PNG']):
        iter+=1
        image = cv2.imread(raw_images_dir + '/' + file)

        (h, w) = image.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

        model.setInput(blob)
        detections = model.forward()

        # Identify each face
        for i in range(0, detections.shape[2]):
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            confidence = detections[0, 0, i, 2]

            # If confidence > 0.5, save it as a separate file
            if (confidence > 0.5):
                count += 1
                frame = image[startY:endY, startX:endX]
                cv2.imwrite(extracted_faces_dir + '/' + str(i) + '_' + file, frame)

print("Extracted " + str(count) + " faces from all images")
print(str(iter) + ' archivos' )

Extracted 5 faces from all images
3 archivos
